# Imports

In [1]:
import os, sys
sys.path.append('..') # Parent directory in path
from time import time, sleep
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import scipy.stats as stats
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
import statsmodels.api as sm

pd.set_option("display.precision", 4)
#import pandas_market_calendars as mcal # NYSE Calendar

import matplotlib.pyplot as plt
plt.style.use('seaborn')
plt.rc("font", **{"size": 14})
plt.rc("figure", **{"figsize": (16,10)})
# import matplotlib.pylab as pl
from matplotlib import cm

# Data
### Pick a specific expiration

In [2]:
all_data = pd.concat([pd.read_hdf(os.path.join("..", "data", "spx_iv_db_1.h5")), 
                      pd.read_hdf(os.path.join("..", "data", "spx_iv_db_2.h5"))])
exp_date = "2017-12-15"
df = all_data[all_data["EXP"]==exp_date].dropna().reset_index(drop=True)
df

,TS,EXP,STRIKE,TYPE,BID_CLOSE,MID_CLOSE,ASK_CLOSE,VOLUME,OPEN_INT,UNDERLYING_PRICE,DIFF,RANK,BUS_DAYS,CAL_DAYS,RATE_OLS,RATE_OPEN_INT,RATE_VOLUME,RATE_MONEYNESS,F_T_OLS,F_T_OPEN_INT,F_T_VOLUME,F_T_MONEYNESS,RATE,F_T,IV_BID,IV_MID,IV_ASK
0,2017-07-03,2017-12-15,2425,C,74.4,75.350,76.30,11,15111,2429.01,4.01,0,116,165,0.0148,0.0148,0.0152,0.0152,2421.7557,2421.7434,2421.7357,2421.7474,0.0148,2421.7434,0.1167,0.1182,0.1196
1,2017-07-03,2017-12-15,2425,P,77.7,78.600,79.50,25,13822,2429.01,4.01,0,116,165,0.0148,0.0148,0.0152,0.0152,2421.7557,2421.7434,2421.7357,2421.7474,0.0148,2421.7434,0.1168,0.1182,0.1196
2,2017-07-03,2017-12-15,2450,P,87.8,88.750,89.70,0,5926,2429.01,20.99,1,116,165,0.0148,0.0148,0.0152,0.0152,2421.7557,2421.7434,2421.7357,2421.7474,0.0148,2421.7434,0.1114,0.1128,0.1143
3,2017-07-03,2017-12-15,2450,C,59.8,60.700,61.60,7,19022,2429.01,20.99,1,116,165,0.0148,0.0148,0.0152,0.0152,2421.7557,2421.7434,2421.7357,2421.7474,0.0148,2421.7434,0.1115,0.1129,0.1143
4,2017-07-03,2017-12-15,2400,P,68.9,69.800,70.70,501,32564,2429.01,29.01,2,116,165,0.0148,0.0148,0.0152,0.0152,2421.7557,2421.7434,2421.7357,2421.7474,0.0148,2421.7434,0.1223,0.1237,0.1251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15278,2017-12-14,2017-12-15,2540,P,0.1,0.200,0.30,20,31619,2652.06,112.06,44,1,1,0.2687,-0.0306,0.3629,0.4772,2654.2863,2654.3611,2654.3095,2654.3193,-0.0306,2654.3611,0.2825,0.3077,0.3253
15279,2017-12-14,2017-12-15,2535,P,0.1,0.200,0.30,73,15733,2652.06,117.06,46,1,1,0.2687,-0.0306,0.3629,0.4772,2654.2863,2654.3611,2654.3095,2654.3193,-0.0306,2654.3611,0.2936,0.3197,0.3378
15280,2017-12-14,2017-12-15,2530,P,0.1,0.200,0.30,16,15516,2652.06,122.06,48,1,1,0.2687,-0.0306,0.3629,0.4772,2654.2863,2654.3611,2654.3095,2654.3193,-0.0306,2654.3611,0.3048,0.3316,0.3502
15281,2017-12-14,2017-12-15,2525,P,0.1,0.200,0.30,1147,67725,2652.06,127.06,50,1,1,0.2687,-0.0306,0.3629,0.4772,2654.2863,2654.3611,2654.3095,2654.3193,-0.0306,2654.3611,0.3159,0.3435,0.3627


In [4]:
# Add ATM_VOLS

ATM_vols = df[df["RANK"]==0]
df = pd.merge(df, ATM_vols[["TS", "TYPE", "IV_BID", "IV_ASK"]], left_on=["TS", "TYPE"], right_on=["TS", "TYPE"], 
              suffixes=(None, "_ATM"), how="left")

# Add useful columns
df["MONEYNESS"] = df["UNDERLYING_PRICE"] - df["STRIKE"]

df["MONEYNESS_F"] = df["STRIKE"]/df["F_T"]
df["LOG_MONEYNESS_F"] = np.log(df["MONEYNESS_F"])
df["LOG_MONEYNESS_F_STANDARD_TIME"] = df["LOG_MONEYNESS_F"] / np.sqrt(df["CAL_DAYS"])

# df["LOG_MONEYNESS_F_STANDARD_STD"] = df["LOG_MONEYNESS_F"] / (np.sqrt(df["CAL_DAYS"])*(df["IV_BID_ATM"]+df["IV_ASK_ATM"])/2)

df["IV_BID_DIFF"] = df["IV_BID"] - df["IV_BID_ATM"]
df["IV_ASK_DIFF"] = df["IV_ASK"] - df["IV_ASK_ATM"]

df["IV_BID_RATIO"] = df["IV_BID"]/df["IV_BID_ATM"]
df["IV_ASK_RATIO"] = df["IV_ASK"]/df["IV_ASK_ATM"]

# Add Greeks
raw_data = pd.concat([pd.read_hdf(os.path.join("..", "data", "vol1.h5")), 
                      pd.read_hdf(os.path.join("..", "data", "vol2.h5"))])

df = pd.merge(df, raw_data[["TS", "EXP", "STRIKE", "TYPE", "IMP_VOLATILITY", "DELTA", "GAMMA", "THETA", "VEGA"]], left_on=["TS", "EXP", "STRIKE", "TYPE"], right_on=["TS", "EXP", "STRIKE", "TYPE"], how="left")

# Add parameters a-e
fit_data = pd.read_excel(os.path.join("..", "processed_data", "iv_parameter.xlsx"), sheet_name=exp_date, index_col=0)
df = pd.merge(df, fit_data.reset_index().rename(columns={"index":"TS"}),left_on=["TS"], right_on=["TS"], how="left")
df

,TS,EXP,STRIKE,TYPE,BID_CLOSE,MID_CLOSE,ASK_CLOSE,VOLUME,OPEN_INT,UNDERLYING_PRICE,DIFF,RANK,BUS_DAYS,CAL_DAYS,RATE_OLS,RATE_OPEN_INT,RATE_VOLUME,RATE_MONEYNESS,F_T_OLS,F_T_OPEN_INT,F_T_VOLUME,F_T_MONEYNESS,RATE,F_T,IV_BID,IV_MID,IV_ASK,IV_BID_ATM,IV_ASK_ATM,MONEYNESS,MONEYNESS_F,LOG_MONEYNESS_F,LOG_MONEYNESS_F_STANDARD_TIME,IV_BID_DIFF,IV_ASK_DIFF,IV_BID_RATIO,IV_ASK_RATIO,IMP_VOLATILITY,DELTA,GAMMA,THETA,VEGA,a,b,c,d,e
0,2017-07-03,2017-12-15,2425,C,74.4,75.350,76.30,11,15111,2429.01,4.01,0,116,165,0.0148,0.0148,0.0152,0.0152,2421.7557,2421.7434,2421.7357,2421.7474,0.0148,2421.7434,0.1167,0.1182,0.1196,0.1167,0.1196,4.01,1.0013,0.0013,0.0001,0.0000,0.0000,1.0000,1.0000,0.0878,0.5625,0.0028,-0.2181,6.4220,0.0656,0.0086,0.0794,-0.0249,-0.0014
1,2017-07-03,2017-12-15,2425,P,77.7,78.600,79.50,25,13822,2429.01,4.01,0,116,165,0.0148,0.0148,0.0152,0.0152,2421.7557,2421.7434,2421.7357,2421.7474,0.0148,2421.7434,0.1168,0.1182,0.1196,0.1168,0.1196,4.01,1.0013,0.0013,0.0001,0.0000,0.0000,1.0000,1.0000,0.1350,-0.4489,0.0018,-0.2230,6.4486,0.0656,0.0086,0.0794,-0.0249,-0.0014
2,2017-07-03,2017-12-15,2450,P,87.8,88.750,89.70,0,5926,2429.01,20.99,1,116,165,0.0148,0.0148,0.0152,0.0152,2421.7557,2421.7434,2421.7357,2421.7474,0.0148,2421.7434,0.1114,0.1128,0.1143,0.1168,0.1196,-20.99,1.0117,0.0116,0.0009,-0.0054,-0.0053,0.9535,0.9560,0.1312,-0.4948,0.0019,-0.2133,6.5015,0.0656,0.0086,0.0794,-0.0249,-0.0014
3,2017-07-03,2017-12-15,2450,C,59.8,60.700,61.60,7,19022,2429.01,20.99,1,116,165,0.0148,0.0148,0.0152,0.0152,2421.7557,2421.7434,2421.7357,2421.7474,0.0148,2421.7434,0.1115,0.1129,0.1143,0.1167,0.1196,-20.99,1.0117,0.0116,0.0009,-0.0052,-0.0054,0.9552,0.9551,0.0878,0.4933,0.0028,-0.2146,6.5012,0.0656,0.0086,0.0794,-0.0249,-0.0014
4,2017-07-03,2017-12-15,2400,P,68.9,69.800,70.70,501,32564,2429.01,29.01,2,116,165,0.0148,0.0148,0.0152,0.0152,2421.7557,2421.7434,2421.7357,2421.7474,0.0148,2421.7434,0.1223,0.1237,0.1251,0.1168,0.1196,29.01,0.9910,-0.0090,-0.0007,0.0055,0.0055,1.0469,1.0461,0.1393,-0.4058,0.0017,-0.2300,6.3201,0.0656,0.0086,0.0794,-0.0249,-0.0014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15278,2017-12-14,2017-12-15,2540,P,0.1,0.200,0.30,20,31619,2652.06,112.06,44,1,1,0.2687,-0.0306,0.3629,0.4772,2654.2863,2654.3611,2654.3095,2654.3193,-0.0306,2654.3611,0.2825,0.3077,0.3253,0.0707,0.0811,112.06,0.9569,-0.0440,-0.0440,0.2118,0.2442,3.9948,4.0130,0.1294,0.0000,0.0000,0.0000,0.0000,0.0649,0.4887,0.0212,2.0000,-0.4663
15279,2017-12-14,2017-12-15,2535,P,0.1,0.200,0.30,73,15733,2652.06,117.06,46,1,1,0.2687,-0.0306,0.3629,0.4772,2654.2863,2654.3611,2654.3095,2654.3193,-0.0306,2654.3611,0.2936,0.3197,0.3378,0.0707,0.0811,117.06,0.9550,-0.0460,-0.0460,0.2229,0.2567,4.1530,4.1672,0.1294,0.0000,0.0000,0.0000,0.0000,0.0649,0.4887,0.0212,2.0000,-0.4663
15280,2017-12-14,2017-12-15,2530,P,0.1,0.200,0.30,16,15516,2652.06,122.06,48,1,1,0.2687,-0.0306,0.3629,0.4772,2654.2863,2654.3611,2654.3095,2654.3193,-0.0306,2654.3611,0.3048,0.3316,0.3502,0.0707,0.0811,122.06,0.9531,-0.0480,-0.0480,0.2341,0.2692,4.3108,4.3210,0.1294,0.0000,0.0000,0.0000,0.0000,0.0649,0.4887,0.0212,2.0000,-0.4663
15281,2017-12-14,2017-12-15,2525,P,0.1,0.200,0.30,1147,67725,2652.06,127.06,50,1,1,0.2687,-0.0306,0.3629,0.4772,2654.2863,2654.3611,2654.3095,2654.3193,-0.0306,2654.3611,0.3159,0.3435,0.3627,0.0707,0.0811,127.06,0.9513,-0.0500,-0.0500,0.2452,0.2816,4.4684,4.4745,0.1294,0.0000,0.0000,0.0000,0.0000,0.0649,0.4887,0.0212,2.0000,-0.4663


[Hyperbola Equation](https://www.desmos.com/calculator/phvqct3f4b):

$$\begin{aligned}
    y &= \sqrt{b^{2}\left(1+\frac{\left(x-c\right)^{2}}{a^{2}}\right)}+d(x-c) + e
\end{aligned}$$


In [4]:
def hyperbola(x, a, b, c, d, e) -> float:
    return (b**2*(1+(x-c)**2/a**2))**0.5+d*(x-c)+e

In [5]:
df['IV_FIT'] = np.sqrt(hyperbola(df["LOG_MONEYNESS_F"], df["a"], df["b"], df["c"], df["d"], df["e"]))
df

,TS,EXP,STRIKE,TYPE,BID_CLOSE,MID_CLOSE,ASK_CLOSE,VOLUME,OPEN_INT,UNDERLYING_PRICE,DIFF,RANK,BUS_DAYS,CAL_DAYS,RATE_OLS,RATE_OPEN_INT,RATE_VOLUME,RATE_MONEYNESS,F_T_OLS,F_T_OPEN_INT,F_T_VOLUME,F_T_MONEYNESS,RATE,F_T,IV_BID,IV_MID,IV_ASK,IV_BID_ATM,IV_ASK_ATM,MONEYNESS,MONEYNESS_F,LOG_MONEYNESS_F,LOG_MONEYNESS_F_STANDARD_TIME,IV_BID_DIFF,IV_ASK_DIFF,IV_BID_RATIO,IV_ASK_RATIO,IMP_VOLATILITY,DELTA,GAMMA,THETA,VEGA,a,b,c,d,e,IV_FIT
0,2017-07-03,2017-12-15,2425,C,74.4,75.350,76.30,11,15111,2429.01,4.01,0,116,165,0.0148,0.0148,0.0152,0.0152,2421.7557,2421.7434,2421.7357,2421.7474,0.0148,2421.7434,0.1167,0.1182,0.1196,0.1167,0.1196,4.01,1.0013,0.0013,0.0001,0.0000,0.0000,1.0000,1.0000,0.0878,0.5625,0.0028,-0.2181,6.4220,0.0656,0.0086,0.0794,-0.0249,-0.0014,0.1180
1,2017-07-03,2017-12-15,2425,P,77.7,78.600,79.50,25,13822,2429.01,4.01,0,116,165,0.0148,0.0148,0.0152,0.0152,2421.7557,2421.7434,2421.7357,2421.7474,0.0148,2421.7434,0.1168,0.1182,0.1196,0.1168,0.1196,4.01,1.0013,0.0013,0.0001,0.0000,0.0000,1.0000,1.0000,0.1350,-0.4489,0.0018,-0.2230,6.4486,0.0656,0.0086,0.0794,-0.0249,-0.0014,0.1180
2,2017-07-03,2017-12-15,2450,P,87.8,88.750,89.70,0,5926,2429.01,20.99,1,116,165,0.0148,0.0148,0.0152,0.0152,2421.7557,2421.7434,2421.7357,2421.7474,0.0148,2421.7434,0.1114,0.1128,0.1143,0.1168,0.1196,-20.99,1.0117,0.0116,0.0009,-0.0054,-0.0053,0.9535,0.9560,0.1312,-0.4948,0.0019,-0.2133,6.5015,0.0656,0.0086,0.0794,-0.0249,-0.0014,0.1126
3,2017-07-03,2017-12-15,2450,C,59.8,60.700,61.60,7,19022,2429.01,20.99,1,116,165,0.0148,0.0148,0.0152,0.0152,2421.7557,2421.7434,2421.7357,2421.7474,0.0148,2421.7434,0.1115,0.1129,0.1143,0.1167,0.1196,-20.99,1.0117,0.0116,0.0009,-0.0052,-0.0054,0.9552,0.9551,0.0878,0.4933,0.0028,-0.2146,6.5012,0.0656,0.0086,0.0794,-0.0249,-0.0014,0.1126
4,2017-07-03,2017-12-15,2400,P,68.9,69.800,70.70,501,32564,2429.01,29.01,2,116,165,0.0148,0.0148,0.0152,0.0152,2421.7557,2421.7434,2421.7357,2421.7474,0.0148,2421.7434,0.1223,0.1237,0.1251,0.1168,0.1196,29.01,0.9910,-0.0090,-0.0007,0.0055,0.0055,1.0469,1.0461,0.1393,-0.4058,0.0017,-0.2300,6.3201,0.0656,0.0086,0.0794,-0.0249,-0.0014,0.1235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15278,2017-12-14,2017-12-15,2540,P,0.1,0.200,0.30,20,31619,2652.06,112.06,44,1,1,0.2687,-0.0306,0.3629,0.4772,2654.2863,2654.3611,2654.3095,2654.3193,-0.0306,2654.3611,0.2825,0.3077,0.3253,0.0707,0.0811,112.06,0.9569,-0.0440,-0.0440,0.2118,0.2442,3.9948,4.0130,0.1294,0.0000,0.0000,0.0000,0.0000,0.0649,0.4887,0.0212,2.0000,-0.4663,0.3097
15279,2017-12-14,2017-12-15,2535,P,0.1,0.200,0.30,73,15733,2652.06,117.06,46,1,1,0.2687,-0.0306,0.3629,0.4772,2654.2863,2654.3611,2654.3095,2654.3193,-0.0306,2654.3611,0.2936,0.3197,0.3378,0.0707,0.0811,117.06,0.9550,-0.0460,-0.0460,0.2229,0.2567,4.1530,4.1672,0.1294,0.0000,0.0000,0.0000,0.0000,0.0649,0.4887,0.0212,2.0000,-0.4663,0.3203
15280,2017-12-14,2017-12-15,2530,P,0.1,0.200,0.30,16,15516,2652.06,122.06,48,1,1,0.2687,-0.0306,0.3629,0.4772,2654.2863,2654.3611,2654.3095,2654.3193,-0.0306,2654.3611,0.3048,0.3316,0.3502,0.0707,0.0811,122.06,0.9531,-0.0480,-0.0480,0.2341,0.2692,4.3108,4.3210,0.1294,0.0000,0.0000,0.0000,0.0000,0.0649,0.4887,0.0212,2.0000,-0.4663,0.3307
15281,2017-12-14,2017-12-15,2525,P,0.1,0.200,0.30,1147,67725,2652.06,127.06,50,1,1,0.2687,-0.0306,0.3629,0.4772,2654.2863,2654.3611,2654.3095,2654.3193,-0.0306,2654.3611,0.3159,0.3435,0.3627,0.0707,0.0811,127.06,0.9513,-0.0500,-0.0500,0.2452,0.2816,4.4684,4.4745,0.1294,0.0000,0.0000,0.0000,0.0000,0.0649,0.4887,0.0212,2.0000,-0.4663,0.3411


In [13]:
all_data

,TS,EXP,STRIKE,TYPE,BID_CLOSE,MID_CLOSE,ASK_CLOSE,VOLUME,OPEN_INT,UNDERLYING_PRICE,DIFF,RANK,BUS_DAYS,CAL_DAYS,RATE_OLS,RATE_OPEN_INT,RATE_VOLUME,RATE_MONEYNESS,F_T_OLS,F_T_OPEN_INT,F_T_VOLUME,F_T_MONEYNESS,RATE,F_T,IV_BID,IV_MID,IV_ASK
0,2017-07-03,2017-07-21,2430,C,16.4,17.05,17.7,374,6981,2429.01,0.99,0,13,18,0.0211,0.0220,0.0138,0.0162,2426.4748,2426.5318,2426.4955,2426.4824,0.0220,2426.5318,0.0823,0.0852,0.0882
1,2017-07-03,2017-07-21,2430,P,19.9,20.50,21.1,281,6410,2429.01,0.99,0,13,18,0.0211,0.0220,0.0138,0.0162,2426.4748,2426.5318,2426.4955,2426.4824,0.0220,2426.5318,0.0824,0.0852,0.0879
2,2017-07-03,2017-07-21,2425,P,18.0,18.50,19.0,373,38136,2429.01,4.01,1,13,18,0.0211,0.0220,0.0138,0.0162,2426.4748,2426.5318,2426.4955,2426.4824,0.0220,2426.5318,0.0854,0.0877,0.0900
3,2017-07-03,2017-07-21,2425,C,19.4,20.05,20.7,194,42228,2429.01,4.01,1,13,18,0.0211,0.0220,0.0138,0.0162,2426.4748,2426.5318,2426.4955,2426.4824,0.0220,2426.5318,0.0848,0.0878,0.0908
4,2017-07-03,2017-07-21,2435,P,22.2,22.80,23.4,394,8399,2429.01,5.99,2,13,18,0.0211,0.0220,0.0138,0.0162,2426.4748,2426.5318,2426.4955,2426.4824,0.0220,2426.5318,0.0802,0.0830,0.0858
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2168401,2022-03-31,2023-03-17,5350,C,40.5,50.35,60.2,140,666,4530.41,819.59,62,242,351,0.0179,0.0183,0.0175,0.0185,4567.4657,4567.5692,4567.3136,4567.7985,0.0183,4567.5692,0.1394,0.1491,0.1581
2168402,2022-03-31,2023-03-17,5375,P,835.8,843.30,850.8,0,2,4530.41,844.59,63,242,351,0.0179,0.0183,0.0175,0.0185,4567.4657,4567.5692,4567.3136,4567.7985,0.0183,4567.5692,0.1442,0.1516,0.1586
2168403,2022-03-31,2023-03-17,5375,C,37.2,46.75,56.3,0,43,4530.41,844.59,63,242,351,0.0179,0.0183,0.0175,0.0185,4567.4657,4567.5692,4567.3136,4567.7985,0.0183,4567.5692,0.1386,0.1485,0.1575
2168404,2022-03-31,2023-03-17,5400,C,36.8,46.80,56.8,0,779,4530.41,869.59,64,242,351,0.0179,0.0183,0.0175,0.0185,4567.4657,4567.5692,4567.3136,4567.7985,0.0183,4567.5692,0.1409,0.1513,0.1609


In [14]:
raw_data

,VOLUME,OPEN_INT,UNDERLYING_PRICE,BID_CLOSE,ASK_CLOSE,IMP_VOLATILITY,DELTA,GAMMA,THETA,VEGA,STRIKE,EXP,TYPE,PRICE,TS
0,0,27.0,2429.01,398.4,404.2,0.0730,1.0000,0.0000,-0.0724,0.0000,2025.0,2017-07-21,C,2429.01,2017-07-03
1,0,0.0,2429.01,393.4,399.2,0.0730,1.0000,0.0000,-0.0726,0.0000,2030.0,2017-07-21,C,2429.01,2017-07-03
2,0,0.0,2429.01,388.4,394.2,0.0730,1.0000,0.0000,-0.0728,0.0000,2035.0,2017-07-21,C,2429.01,2017-07-03
3,0,1.0,2429.01,383.4,389.2,0.0730,1.0000,0.0000,-0.0730,0.0000,2040.0,2017-07-21,C,2429.01,2017-07-03
4,0,6.0,2429.01,378.4,384.3,0.0730,1.0000,0.0000,-0.0732,0.0000,2045.0,2017-07-21,C,2429.01,2017-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407306,0,0.0,4530.41,774.7,789.3,0.1125,-0.9115,0.0003,-0.0929,7.0422,5300.0,2023-03-17,P,4530.41,2022-03-31
2407307,0,1.0,4530.41,794.8,809.5,0.1072,-0.9287,0.0003,-0.0707,5.9653,5325.0,2023-03-17,P,4530.41,2022-03-31
2407308,0,24.0,4530.41,815.1,830.0,0.0994,-0.9490,0.0002,-0.0437,4.5633,5350.0,2023-03-17,P,4530.41,2022-03-31
2407309,0,2.0,4530.41,835.8,850.8,0.0994,-0.9537,0.0002,-0.0385,4.2126,5375.0,2023-03-17,P,4530.41,2022-03-31


In [16]:
df = pd.merge(all_data, raw_data[["TS", "EXP", "STRIKE", "TYPE", "IMP_VOLATILITY", "DELTA", "GAMMA", "THETA", "VEGA"]], left_on=["TS", "EXP", "STRIKE", "TYPE"], right_on=["TS", "EXP", "STRIKE", "TYPE"], how="left")

,TS,EXP,STRIKE,TYPE,BID_CLOSE,MID_CLOSE,ASK_CLOSE,VOLUME,OPEN_INT,UNDERLYING_PRICE,DIFF,RANK,BUS_DAYS,CAL_DAYS,RATE_OLS,RATE_OPEN_INT,RATE_VOLUME,RATE_MONEYNESS,F_T_OLS,F_T_OPEN_INT,F_T_VOLUME,F_T_MONEYNESS,RATE,F_T,IV_BID,IV_MID,IV_ASK,IMP_VOLATILITY,DELTA,GAMMA,THETA,VEGA
0,2017-07-03,2017-07-21,2430,C,16.4,17.05,17.7,374,6981,2429.01,0.99,0,13,18,0.0211,0.0220,0.0138,0.0162,2426.4748,2426.5318,2426.4955,2426.4824,0.0220,2426.5318,0.0823,0.0852,0.0882,0.0730,0.5085,0.0103,-0.4882,2.1112
1,2017-07-03,2017-07-21,2430,P,19.9,20.50,21.1,281,6410,2429.01,0.99,0,13,18,0.0211,0.0220,0.0138,0.0162,2426.4748,2426.5318,2426.4955,2426.4824,0.0220,2426.5318,0.0824,0.0852,0.0879,0.0983,-0.4918,0.0077,-0.5551,2.1112
2,2017-07-03,2017-07-21,2425,P,18.0,18.50,19.0,373,38136,2429.01,4.01,1,13,18,0.0211,0.0220,0.0138,0.0162,2426.4748,2426.5318,2426.4955,2426.4824,0.0220,2426.5318,0.0854,0.0877,0.0900,0.1002,-0.4542,0.0075,-0.5661,2.0977
3,2017-07-03,2017-07-21,2425,C,19.4,20.05,20.7,194,42228,2429.01,4.01,1,13,18,0.0211,0.0220,0.0138,0.0162,2426.4748,2426.5318,2426.4955,2426.4824,0.0220,2426.5318,0.0848,0.0878,0.0908,0.0730,0.5600,0.0102,-0.4877,2.0878
4,2017-07-03,2017-07-21,2435,P,22.2,22.80,23.4,394,8399,2429.01,5.99,2,13,18,0.0211,0.0220,0.0138,0.0162,2426.4748,2426.5318,2426.4955,2426.4824,0.0220,2426.5318,0.0802,0.0830,0.0858,0.0969,-0.5306,0.0078,-0.5415,2.1055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2168401,2022-03-31,2023-03-17,5350,C,40.5,50.35,60.2,140,666,4530.41,819.59,62,242,351,0.0179,0.0183,0.0175,0.0185,4567.4657,4567.5692,4567.3136,4567.7985,0.0183,4567.5692,0.1394,0.1491,0.1581,0.1528,0.1519,0.0003,-0.2295,10.4309
2168402,2022-03-31,2023-03-17,5375,P,835.8,843.30,850.8,0,2,4530.41,844.59,63,242,351,0.0179,0.0183,0.0175,0.0185,4567.4657,4567.5692,4567.3136,4567.7985,0.0183,4567.5692,0.1442,0.1516,0.1586,0.0994,-0.9537,0.0002,-0.0385,4.2126
2168403,2022-03-31,2023-03-17,5375,C,37.2,46.75,56.3,0,43,4530.41,844.59,63,242,351,0.0179,0.0183,0.0175,0.0185,4567.4657,4567.5692,4567.3136,4567.7985,0.0183,4567.5692,0.1386,0.1485,0.1575,0.1521,0.1434,0.0003,-0.2198,10.0380
2168404,2022-03-31,2023-03-17,5400,C,36.8,46.80,56.8,0,779,4530.41,869.59,64,242,351,0.0179,0.0183,0.0175,0.0185,4567.4657,4567.5692,4567.3136,4567.7985,0.0183,4567.5692,0.1409,0.1513,0.1609,0.1549,0.1415,0.0003,-0.2218,9.9460


In [18]:
df.to_hdf(os.path.join("..", "processed_data", "spx_iv_db.h5"), 'data')